# Data extraction from multiple text files and output a single excel file

### Import Libraries

In [ ]:
#Import Lib
import re
import glob
import pandas as pd
from pandas import ExcelWriter

### Code for search needed data from multiple .txt files

In [4]:
#Text file folder location
file_list = glob.glob('C:/Users/Site Fuel/Desktop/Codes/Colab Notebooks/txt sample files/*.txt')

#Defined Veriable
new_file_list=[]
ip_addresses = []
manufacturer = []
ntp_addresses = []
syslog_addresses = []
tacacs_addresses_1 = []
tacacs_addresses_2 = []
NAC = []
snmp = []
version = []

#For Loop
for i in file_list:
    f = open(i,"r")
    text = f.read()
    
    #File name edit
    ifi = i.replace('C:/Users/Site Fuel/Desktop/Codes/Colab Notebooks/txt sample files','')
    new_file_list.append(ifi)
    
    #IP addresses
    ip = re.search(r"\d+\.\d+\.\d+\.\d+", i)
    try:
        ip_addresses.append(ip.group())
    except: 
        ip_addresses.append("None")
    
    #manufacturer
    make_1 = re.search(r'Cisco', text)
    make_2 = re.search(r'Brocade', text)
    make_3 = re.search(r'Juniper', text)
    try:
        manufacturer.append(make_1.group(0).split()[0])
    except:
        try:
            manufacturer.append(make_2.group(0).split()[0])
        except:
            try:
                manufacturer.append(make_3.group(0).split()[0])
            except:
                  manufacturer.append("None")
                    
    #NTP addresses
    ntp_1 = re.search(r'ntp server \d+\.\d+\.\d+\.\d+', text)
    ntp_2 = re.search(r'ntp\n.server \d+\.\d+\.\d+\.\d+', text)
    ntp_3 = re.search(r'server \d+\.\d+\.\d+\.\d+ prefer;', text)
    try:
        ntp_addresses.append(ntp_1.group(0).split()[2])
    except:
        try:
            ntp_addresses.append(ntp_2.group(0).split()[2])
        except:
            try:
                ntp_addresses.append(ntp_3.group(0).split()[1])  
            except:
                ntp_addresses.append("None")
    
    #SYSLOG Addresses
    syslog_1 = re.search(r'logging host \d+\.\d+\.\d+\.\d+', text)
    syslog_2  = re.search(r'notifications\nlogging \d+\.\d+\.\d+\.\d+', text)
    try:
        syslog_addresses.append(syslog_1.group(0).split()[2])   
    except:
        try:
            syslog_addresses.append(syslog_2.group(0).split()[2]) 
        except:
            syslog_addresses.append("None")
    
    #TACACS_addresses_1
    tacacs_1 = re.search(r'tacacs-server host \d+\.\d+\.\d+\.\d+', text)
    tacacs_2  = re.search(r'tacplus-server \{(.*?)\}', text, re.DOTALL)
    try:
        tacacs_addresses_1.append(tacacs_1.group(0).split()[2]) 
    except:
        try:
            tacacs_addresses_1.append(tacacs_2.group(0).split()[2]) 
        except:
            tacacs_addresses_1.append("None")
           
    #TACACS_addresses_2
    tacacs_3 = re.findall(r'(tacacs-server host \d+\.\d+\.\d+\.\d+)', text)
    tacacs_4  = re.search(r'tacplus-server \{(.*?)\}', text, re.DOTALL)
    try:
        tacacs_addresses_2.append(tacacs_3[1].split()[2]) 
    except:
        try:
            tacacs_addresses_2.append(tacacs_4.group(0).split()[2]) 
        except:
            tacacs_addresses_2.append("None")
    
    #NAC_Dot1x
    nac = re.search(r'dot1x', text)
    try:
        NAC.append(nac.group(0).split()[0])
    except:
        NAC.append("None")
    
    #SNMP
    snmp_1 = re.search(r'snmp-server group GROUP1 v3', text)
    snmp_2 = re.search(r'snmp-server host \d+\.\d+\.\d+\.\d+ version v3', text)
    try:
        snmp.append(snmp_1.group(0).split()[3])
    except:
        try:
            snmp.append(snmp_2.group(0).split()[4])
        except:
            snmp.append("None")
    
    #Version
    version_1  = re.search(r'\SW Version(.*?)\-M', text, re.DOTALL)
    try:
        version.append(version_1.group(0).split()[13]) 
    except:
        version.append("None")

### Code for dataframe and convert into .xlsx file

In [5]:
#Data_Frame for xls
new_dict = {'File_Name': new_file_list, 'IP_address': ip_addresses, 'manufacturer' : manufacturer, 'NTP_server': ntp_addresses, 'NAC Server' : NAC, 'SysLog' : syslog_addresses, 'TACACS_1' : tacacs_addresses_1, 'TACACS_2' : tacacs_addresses_2}
data_frame= pd.DataFrame(new_dict)
print(data_frame)

# Data_Frame to EXCEL
writer = ExcelWriter('C:/Users/Site Fuel/Desktop/Codes/Colab Notebooks/Output.xlsx')
data_frame.to_excel(writer,'Sheet1')
writer.save()

                                       File_Name     IP_address manufacturer  \
0                           \172.17.51.7 (2).txt    172.17.51.7        Cisco   
1                              \172.18.133.7.txt   172.18.133.7      Brocade   
2    \AOKOLA01SW_RENE1_S2960_7_172.18.124.10.txt  172.18.124.10        Cisco   
3                  \AOMAD34SW_2_172.18.133.7.txt   172.18.133.7         None   
4  \PNE-DISPENBLD-GF-SW3750-02_172.22.72.224.txt  172.22.72.224         None   
5  \PNE-E17GF-COL17-SW2960_172.22.144.37-new.txt  172.22.144.37        Cisco   
6      \PNE-E17GF-COL17-SW2960_172.22.144.37.txt  172.22.144.37        Cisco   
7   \PNE-GDC-SWHP2920-stack03_172.18.4.6 (2).txt     172.18.4.6         None   
8        \PNE_E16-17_FF_Col_15_172.22.144.20.txt  172.22.144.20      Juniper   

      NTP_server NAC Server          SysLog       TACACS_1       TACACS_2  
0   172.31.9.241       None  172.27.158.186  172.22.66.101  172.22.66.102  
1  172.24.47.207      dot1x     128.9.51.48  17